# Лабораторная 3: Рекомендации и категориальные признаки

### Выполнила: Егорова Вера

## Описание данных

В этой лабораторной работе будет рассмотрена задача предсказания оценки, которую поставит пользователь фильму. Особенность этой задачи в том, что вам предстоит работать с объектами обучающей выборки, которые описаны категориальными признаками, принимающими большое число значений (пример: идентификатор пользователя, идентификатор фильма).

![](http://i.imgur.com/vHUVoWw.png)

Мы будем работать с датасетом [MovieLens + IMDb/Rotten Tomatoes](http://files.grouplens.org/datasets/hetrec2011/hetrec2011-movielens-2k-v2.zip) ([описание](http://files.grouplens.org/datasets/hetrec2011/hetrec2011-movielens-readme.txt)). Набор содержит данные о предпочтениях пользователей сервиса рекомендации кинофильмов [MovieLens](http://www.movielens.org/). Пользователь ставит рейтинг фильму в интервале от 1 до 5. Оценки записаны в файле *user_ratedmovies.dat*, остальные файлы содержат дополнительную информацию о фильмах, которую можно использовать как признаки. Заметьте: кроме оценок (и тегов), про пользователя ничего не известно.

На основании этих данных необходимо построить модель, предсказывающую оценку пользователя фильму, который он еще не смотрел.

## Оценка качества

Выберем некоторого пользователя $u$ и обозначим известные для него рейтинги за $R^u$. В качестве тестовых рейтингов этого пользователя $R^u_{test}$ рассмотрим три рейтинга, поставленные последними по времени. Остальные известные рейтинги будут составлять обучающую выборку $R^u_{train}$. Тогда все известные рейтинги можно представить как $R^u=R^u_{train}\cup R^u_{test}$. Отсутствующие оценки обозначим за $R^u_{unknown}$. Объединив эти наборы для всех пользователей, получим наборы $R_{train}$, $R_{test}$ и $R_{unknown}$.

Для измерения качества рекомендаций используйте две метрики, описанные ниже.

#### RMSE

Метрика [RMSE](https://en.wikipedia.org/wiki/Root-mean-square_deviation) вычисляется следующим образом:
$$ RMSE = \sqrt{ \frac{1}{\left|R_{test}\right|} \sum_{(u,i) \in R_{test}} (r_{ui} - \hat{r}_{ui})^2 },$$
где $r_{ui}$ — наблюдаемая (правильная) оценка, а $\hat{r}_{ui}$ — оценка, предсказанная моделью.

Метрика RMSE предназначена для оценки точности предсказания, ее удобно оптимизировать напрямую. Однако RMSE не совсем правильно использовать при оценке качества рекомендаций: RMSE одинаково штрафует точность предсказания оценок фильмам с большим значением предпочтения (которые попадут в блок рекомендаций) и фильмам с малым значением предпочтения (длинный хвост из нерелевантных фильмов).

#### MAP

Для оценки качества рекомендаций можно использовать метрику качества ранжирования. Для этого для каждого пользователя $u$ предскажем оценку для всех фильмов из $R^u_{test}$ и $R^u_{unknown}$ и отсортируем эти фильмы по убыванию предсказанного рейтинга. Ожидается, что хороший алгоритм должен выдать релевантные фильмы вверху списка. Обозначим позиции объектов в этом списке за $k^u_i$.

Назовем релевантными те фильмы, которые входят в $R^u_{test}$ и имеют оценку $\ge 3$. Обозначим их за $Rel^u$. Тогда можно считать следующую метрику качества рекомендаций для одного пользователя:

$$AP^u=\frac{1}{|Rel^u|} \sum_{(u,i) \in Rel^u} \frac{1}{k^u_i}.$$

Усреднив значение этой метрики по всем пользователями, мы получим окончательное значение метрики $MAP$. Пользователей без релевантных фильмов в тестовой выборке можно не учитывать.

# Задания

#### Коллаборативная фильтрация (4 балла)

Для построения рекомендаций будем использовать алгоритмы на основе матричных разложений, в том числе предоставляемые библиотекой [Graphlab Create](https://dato.com/products/create/). Для оценки качества используйте две метрики, описанные выше. Не забывайте делать аргументированные выводы на основании полученных результатов.
 1. Постройте рекомендации с помощью PureSVD. Для этого создайте разреженную матрицу пользователи-фильмы, где в каждой ячейке стоит рейтинг, если он известен, или ноль, если неизвестен. Разложив эту матрицу с помощью разреженного SVD и восстановив ее, можно получить предсказания рейтингов для всех пар пользователь-объект. Оцените качество работы  алгоритма в зависимости от выбранного ранга разложения.
 2. Далее будем рассматривать как работают рекомендации из библиотеки Graphlab Create:
  - Проведите аналогичный эксперимент для [алгоритма рекомендаций из GraphLab Create](https://dato.com/products/create/docs/generated/graphlab.recommender.ranking_factorization_recommender.RankingFactorizationRecommender.html), сравните его с PureSVD.
  - Попробуйте использовать этот метод не с квадратичными потерями, а логистическими, т.е. решая задачу бинарной классификации (отделение хороших фильмов от плохих). Для этого надо использовать параметр *binary_target*. Дало ли это прирост в качестве?
  - Поэксперементируйте с другими параметрами алгоритма и сделайте выводы, как их изменение влияет на качество предсказания.

Загрузим базу данных со всеми оценками фильмов, проставленными различными пользователями.

In [1]:
from collections import defaultdict
import graphlab
import pandas as pd

In [2]:
movies_df = pd.read_csv("data/movies.dat", sep = '\t')
print 'Movies db size: {}'.format(len(movies_df))
movies_df.head()

Movies db size: 10197


,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtAllCriticsRating,rtAllCriticsNumReviews,rtAllCriticsNumFresh,...,rtAllCriticsScore,rtTopCriticsRating,rtTopCriticsNumReviews,rtTopCriticsNumFresh,rtTopCriticsNumRotten,rtTopCriticsScore,rtAudienceRating,rtAudienceNumRatings,rtAudienceScore,rtPictureURL
0,1,Toy story,114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995,toy_story,9,73,73,...,100,8.5,17,17,0,100,3.7,102338,81,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995,1068044-jumanji,5.6,28,13,...,46,5.8,5,2,3,40,3.2,44587,61,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,107050,Dos viejos gru�ones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993,grumpy_old_men,5.9,36,24,...,66,7,6,5,1,83,3.2,10489,66,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995,waiting_to_exhale,5.6,25,14,...,56,5.5,11,5,6,45,3.3,5666,79,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,113041,Vuelve el padre de la novia (Ahora tambi�n abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995,father_of_the_bride_part_ii,5.3,19,9,...,47,5.4,5,1,4,20,3,13761,64,http://content8.flixster.com/movie/25/54/25542...


In [3]:
movies_df = pd.read_csv("data/movies.dat", sep = '\t')
rated_movies = pd.read_csv("data/user_ratedmovies.dat", sep = '\t')

print 'Rated movies db size: {}'.format(len(rated_movies))
movies_df.fillna(0, inplace=True)
rated_movies.fillna(0, inplace=True)
rated_movies.head()

Rated movies db size: 855598


,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,75,3,1.0,29,10,2006,23,17,16
1,75,32,4.5,29,10,2006,23,23,44
2,75,110,4.0,29,10,2006,23,30,8
3,75,160,2.0,29,10,2006,23,16,52
4,75,163,4.0,29,10,2006,23,29,30


Для каждой оценки сформируем временную метку timestamp

In [4]:
import datetime, time
def timestamp(x):
    date = datetime.datetime(int(x['date_year']), int(x['date_month']), 
                 int(x['date_day']), int(x['date_hour']), 
                 int(x['date_minute']), int(x['date_second']))
    stamp = time.mktime(date.timetuple())
    return int(stamp)

rated_movies['timestamp'] = rated_movies.apply(timestamp, axis=1)
rated_movies.head()

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second,timestamp
0,75,3,1.0,29,10,2006,23,17,16,1162142236
1,75,32,4.5,29,10,2006,23,23,44,1162142624
2,75,110,4.0,29,10,2006,23,30,8,1162143008
3,75,160,2.0,29,10,2006,23,16,52,1162142212
4,75,163,4.0,29,10,2006,23,29,30,1162142970


Построим множества $R_{train}$ и $R_{test}$

In [6]:
test_df = {}
train_df = {}
ranked_df = defaultdict(list)
users = rated_movies['userID'].unique()
movies = movies_df['id']
mean_user_rating = {}
grouped_df = rated_movies.groupby('userID')
for user_id in users:
    group_df = grouped_df.get_group(user_id).sort('timestamp', ascending=False)
    test_df[user_id] = group_df[:3]
    train_df[user_id] = group_df[3:]
    ranked_df[user_id] = list(group_df['movieID'])

print train_df[75].head()
print test_df[75]
print ranked_df[75]

    userID  movieID  rating  date_day  date_month  date_year  date_hour  \
39      75     4993     3.5        29          10       2006         23   
34      75     2959     4.5        29          10       2006         23   
2       75      110     4.0        29          10       2006         23   
18      75     1233     4.0        29          10       2006         23   
41      75     5833     2.5        29          10       2006         23   

    date_minute  date_second   timestamp  
39           30           34  1162143034  
34           30           12  1162143012  
2            30            8  1162143008  
18           30            5  1162143005  
41           29           51  1162142991  
    userID  movieID  rating  date_day  date_month  date_year  date_hour  \
29      75     2571     4.5        29          10       2006         23   
42      75     5952     3.5        29          10       2006         23   
48      75     7153     3.5        29          10       2006      

/Library/Python/2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


Посчитаем средний рейтинг для каждого фильма и отцентрируем все рейтинги в множестве $R_{train}$

###PureSVD

Сформируем разреженную матрицу пользователи-фильмы:

In [15]:
movies = {}
inverse_movies = {}
for movie in movies_df['id'].unique():
    movies[movie] = len(movies)

users = {}
inverse_users = {}
for user in rated_movies['userID'].unique():
    users[user] = len(users)

print 'Movies number: {}'.format(len(movies))
print 'Users number: {}'.format(len(users))

Movies number: 10197
Users number: 2113


In [17]:
import numpy as np

matrix = np.zeros((len(users), len(movies)))
for user, data in train_df.items():
    for row in data.iterrows():
        userID = users[user]
        movieID = movies[row[1].movieID]
        matrix[userID][movieID] = row[1].rating
sparse_matrix = scipy.sparse.dok_matrix(matrix)
print sparse_matrix.shape

Разложим полученную матрицу с помощью SVD разложения.

In [178]:
import scipy
from scipy.sparse.linalg import svds
U, s, V = svds(sparse_matrix, k=30)
print U.shape, s.shape, V.shape

(2113L, 30L) (30L,) (30L, 10197L)


In [262]:
%%time
def train(marix, rank):
    U, s, V = svds(sparse_matrix, rank)
    return np.matrix(U) * np.diag(s) * np.matrix(V)

def calculate_RMSE(test_df, result_mat):
    RMSE = 0.0
    total = 0
    for user, data in test_df.items():
        for row in data.iterrows():
            userID = users[user]
            movieID = movies[row[1].movieID]
            real_rating = row[1].rating
            predicted_rating = result_mat[userID, movieID]
            RMSE += (real_rating - predicted_rating)**2
            total += 2
    return (RMSE / total)**(0.5)
   
res_mat = train(sparse_matrix, 15)
print 'RMSE: {0:.3f}'.format(calculate_RMSE(test_df, res_mat))        

RMSE: 2.214
Wall time: 52.2 s


In [250]:
def calculate_MAP(train_df, test_df, result_mat, movies, users):
    all_moives = set(movies_df['id'])
    ap = []
    total = 0
    for user in rated_movies['userID'].unique():
        userID = users[user]
        predicted = []
        for movie in all_moives-set(train_df[user]['movieID']):
            movieID = movies[movie]
            predicted.append((movie, result_mat[userID, movieID]))
        position = {x[0]: k for k, x in enumerate(sorted(predicted, key=lambda x:x[1], reverse=True)) if x[1] >= 3.}
        rel = [1./(position[x] + 1) for x in test_df[user]['movieID'] if x in position.keys()]
        if len(rel) > 0:
            ap.append(sum(rel)/len(rel))
    return np.mean(ap)
            
    grouped_df = test_df.groupby('userID')

print 'MAP: {0:.3f}'.format(calculate_MAP(train_df, test_df, res_mat,  movies, users))        

MAP: 0.456


Судя по значениям метрик, это не самы лучший алгоритм. Меня долго смущала такая большая ошибка RMSE, я перепробовала разные алгоритмы для SVD разложения, делала различные преобразования с матрицей пользователей-фильмов (учитывала средние оценки пользователей и фильмов), но в итоге лучше результата для Pure SVD я не получила.

In [255]:
ranks = [10, 15, 25, 50, 100]
df = pd.DataFrame(columns=['rmse', 'map'], index=ranks)
for rank in ranks:
    res_mat = train(sparse_matrix, rank)
    rmse_score = calculate_RMSE(test_df, res_mat)
    map_score = calculate_MAP(train_df, test_df, res_mat, movies, users)
    df.loc[rank] = pd.Series({'rmse':rmse_score, 'map':map_score})
df = pd.DataFrame.from_csv('pure_svd.res', sep='\t')

,rmse,map
10,2.226473,0.4563842
15,2.214206,0.5094019
25,2.202443,0.5138819
50,2.202367,0.5593529
100,2.228019,0.6127663


## Рекомендации из библиотеки Graphlab Create

2. Далее будем рассматривать как работают рекомендации из библиотеки Graphlab Create:
  - Проведите аналогичный эксперимент для [алгоритма рекомендаций из GraphLab Create](https://dato.com/products/create/docs/generated/graphlab.recommender.ranking_factorization_recommender.RankingFactorizationRecommender.html), сравните его с PureSVD.
  - Попробуйте использовать этот метод не с квадратичными потерями, а логистическими, т.е. решая задачу бинарной классификации (отделение хороших фильмов от плохих). Для этого надо использовать параметр *binary_target*. Дало ли это прирост в качестве?
  - Поэксперементируйте с другими параметрами алгоритма и сделайте выводы, как их изменение влияет на качество предсказания.

In [297]:
def prepare_data(data, with_rating=True, verbose=False, binary=False):
    frame_data = defaultdict(list)
    for user, data in data.items():
        frame_data['item_id'] += list(data['movieID'])
        if with_rating:
            frame_data['rating'] += list(data['rating']) if not binary else ([1.] *len(data['movieID']))
        frame_data['user_id'] += ([user] * len(data['movieID']))
        
    if verbose:
        print 'User id:', frame_data['user_id'][:5]
        print 'Movie id:', frame_data['item_id'][:5]
        if with_rating:
            print 'Rating:', frame_data['rating'][:5]
    return graphlab.SFrame(frame_data)
train_data = prepare_data(train_df, verbose=True)

User id: [8194, 8194, 8194, 8194, 8194]
Movie id: [3793, 49647, 49793, 48774, 224]
Rating: [4.0, 4.0, 3.5, 4.0, 2.5]


In [275]:
def train_model(sf, rank):
    model = graphlab.ranking_factorization_recommender.create(sf, target='rating',solver='sgd',
                                                              num_factors=rank, max_iterations=100, verbose=False)
    return model
model=train_model(train_data, 15)

PROGRESS: Recsys training: model = ranking_factorization_recommender


В логе обучения видим, что на обучающей выборке метрика $RMSE=0.749001$. Посмотрим, как наша модель предсказывает неизвестные рейтинги.

In [276]:
test_data=prepare_data(test_df, verbose=True)
result = model.predict(test_data)
print 'Predicted rating:',result[:5]
print 'RMSE:', model.evaluate_rmse(test_data, target='rating')['rmse_overall']

User id: [8194, 8194, 8194, 16389, 16389]
Movie id: [44191, 5349, 2640, 57223, 3702]
Rating: [3.0, 4.0, 3.5, 3.5, 3.0]
Predicted rating: [2.8196000247740742, 4.149938800885677, 3.8065895825171467, 1.6796073704504964, 2.8953903294646737]
RMSE: 1.23401262484


На тестовой выобрки ошибка модели больше. Но она до сих пор работает значительно лучше предыдущего метода. Теперь посмотрим, что покажет метрика MAP.

In [277]:
def calculate_map(model, test_df, ranked_df, movies, users, verbose=False):
    ap = []
    for user in users:
        test_movies = list(test_df[user]['movieID'])
        movies = [movie for movie in movies if movie not in ranked_df[user]] + test_movies
        test_data = prepare_data({user:{'movieID': movies}}, with_rating=False, verbose=verbose)
        predicted = {movie: (i, rank) for i, (movie, rank) in enumerate
                             (sorted(zip(movies, model.predict(test_data)), key=lambda x: x[1], reverse=True))}
        rel = [1./(predicted[x][0]+1) for x in test_movies if predicted[x][1] >=3]
        if verbose: print rel
        ap_u = 0 if not rel else sum(rel)/len(rel)
        if verbose: print ap_u
        ap.append(ap_u)
    return sum(ap) / len(ap)
        

users = rated_movies['userID'].unique()
movies = movies_df['id']
MAP = calculate_map(model, test_df, ranked_df, movies, users, verbose=False)
print 'MAP:', MAP

MAP: 0.399657188642


Метрика MAP подтверждает, что модель от graphlab работает лучше pure svd. Для чистой совести проведем еще один эксперимент (как и для pure svd).

In [292]:
ranks = [10, 15, 25, 50, 100]
df_1 = pd.DataFrame(columns=['g_rmse', 'g_map'], index=ranks)
train_data = prepare_data(train_df)
test_data = prepare_data(test_df)
for rank in ranks:
    print rank
    model = train_model(train_data, rank)
    rmse_score = model.evaluate_rmse(test_data, target='rating')['rmse_overall']
    map_score = calculate_map(model, test_df, ranked_df, movies, users)
    df_1.loc[rank] = pd.Series({'g_rmse':rmse_score, 'g_map':map_score})

pd.concat([df, df_1], axis=1)

,rmse,map,g_rmse,g_map
10,2.226473,0.4563842,1.165691,0.3961264
15,2.214206,0.5094019,1.227364,0.401565
25,2.202443,0.5138819,1.240567,0.4113039
50,2.202367,0.5593529,1.408441,0.4087704
100,2.228019,0.6127663,1.658263,0.4106842


В таблице выше приведено сравнение алгоритмов Pure SVD и Graphlab Create. Наилучшие показатели метрик наблюдаются у второго алгоритма.

Посмотрим, как повлияет на качетсво использование тогоже метода но с логистическими потерями.

In [298]:
train_data = prepare_data(train_df, binary=True) 
model = graphlab.ranking_factorization_recommender.create(train_data, target='rating',solver='sgd',
                                                           num_factors=15, max_iterations=100, 
                                                           verbose=True, binary_target=True)

PROGRESS: Recsys training: model = ranking_factorization_recommender
PROGRESS: Preparing data set.
PROGRESS:     Data has 849259 observations with 2113 users and 10084 items.
PROGRESS:     Data prepared in: 0.655065s
PROGRESS: Training ranking_factorization_recommender for recommendations.
PROGRESS: +--------------------------------+--------------------------------------------------+----------+
PROGRESS: | Parameter                      | Description                                      | Value    |
PROGRESS: +--------------------------------+--------------------------------------------------+----------+
PROGRESS: | num_factors                    | Factor Dimension                                 | 15       |
PROGRESS: | regularization                 | L2 Regularization on Factors                     | 1e-009   |
PROGRESS: | solver                         | Solver used for training                         | sgd      |
PROGRESS: | linear_regularization          | L2 Regularization on L

Видим, что на тренировочных данных ошибка меньше чем во всех предыдущих экспериментах и состовляет $0.42$.

In [312]:
test_data=prepare_data(test_df, verbose=False, binary=True)
print 'RMSE:', model.evaluate_rmse(test_data, target='rating')['rmse_overall']

RMSE: 0.531227890484


На тестовых данных ошибка тоже мала. На данных момент это самая удачная модель из всех обученных. Теперь посмотрим, как влияют на качество предсказания другие параметры алгоримта. Сначала испробуем все возможные решатели.

In [300]:
train_data = prepare_data(train_df) 
model = graphlab.ranking_factorization_recommender.create(train_data, target='rating',solver='adagrad',
                                                           num_factors=15, max_iterations=100, 
                                                           verbose=True)

PROGRESS: Recsys training: model = ranking_factorization_recommender
PROGRESS: Preparing data set.
PROGRESS:     Data has 849259 observations with 2113 users and 10084 items.
PROGRESS:     Data prepared in: 0.738904s
PROGRESS: Training ranking_factorization_recommender for recommendations.
PROGRESS: +--------------------------------+--------------------------------------------------+----------+
PROGRESS: | Parameter                      | Description                                      | Value    |
PROGRESS: +--------------------------------+--------------------------------------------------+----------+
PROGRESS: | num_factors                    | Factor Dimension                                 | 15       |
PROGRESS: | regularization                 | L2 Regularization on Factors                     | 1e-009   |
PROGRESS: | solver                         | Solver used for training                         | adagrad  |
PROGRESS: | linear_regularization          | L2 Regularization on L

In [314]:
test_data=prepare_data(test_df, verbose=False)
print 'RMSE:', model.evaluate_rmse(test_data, target='rating')['rmse_overall']

RMSE: 3.23432813831


Вывод: Решатели adagrad и sgd дают примерно одинаковое качество.

In [303]:
model = graphlab.ranking_factorization_recommender.create(train_data, target='rating',solver='ials',
                                                           num_factors=15, max_iterations=100, 
                                                           verbose=True)

PROGRESS: Recsys training: model = ranking_factorization_recommender
PROGRESS: Preparing data set.
PROGRESS:     Data has 849259 observations with 2113 users and 10084 items.
PROGRESS:     Data prepared in: 0.869604s
PROGRESS: Training ranking_factorization_recommender for recommendations.
PROGRESS: +------------------------------+--------------------------------------------------+----------+
PROGRESS: | Parameter                    | Description                                      | Value    |
PROGRESS: +------------------------------+--------------------------------------------------+----------+
PROGRESS: | num_factors                  | Factor Dimension                                 | 15       |
PROGRESS: | regularization               | L2 Regularization on Factors                     | 1e-009   |
PROGRESS: | max_iterations               | Maximum Number of Iterations                     | 100      |
PROGRESS: | solver                       | Solver used for training            

Решателю ials не хватило 100 итераций:( Качество даже хуже, чем у нашего Pure svd.

In [304]:
print 'RMSE:', model.evaluate_rmse(test_data, target='rating')['rmse_overall']

RMSE: 3.38621657772


А что если установить binary_target=True?

In [305]:
train_data = prepare_data(train_df, binary=True) 
model = graphlab.ranking_factorization_recommender.create(train_data, target='rating',solver='adagrad',
                                                           num_factors=15, max_iterations=100, 
                                                           verbose=True, binary_target=True)

PROGRESS: Recsys training: model = ranking_factorization_recommender
PROGRESS: Preparing data set.
PROGRESS:     Data has 849259 observations with 2113 users and 10084 items.
PROGRESS:     Data prepared in: 0.756288s
PROGRESS: Training ranking_factorization_recommender for recommendations.
PROGRESS: +--------------------------------+--------------------------------------------------+----------+
PROGRESS: | Parameter                      | Description                                      | Value    |
PROGRESS: +--------------------------------+--------------------------------------------------+----------+
PROGRESS: | num_factors                    | Factor Dimension                                 | 15       |
PROGRESS: | regularization                 | L2 Regularization on Factors                     | 1e-009   |
PROGRESS: | solver                         | Solver used for training                         | adagrad  |
PROGRESS: | linear_regularization          | L2 Regularization on L

In [306]:
test_data=prepare_data(test_df, verbose=False, binary=True)
print 'RMSE:', model.evaluate_rmse(test_data, target='rating')['rmse_overall']

RMSE: 0.564590083264


Наблюдение: adagrad незначительно превзашел в качестве sgd.

In [307]:
model = graphlab.ranking_factorization_recommender.create(train_data, target='rating',solver='ials',
                                                           num_factors=15, max_iterations=100, 
                                                           verbose=True, binary_target=True)

PROGRESS: Recsys training: model = ranking_factorization_recommender
PROGRESS: Preparing data set.
PROGRESS:     Data has 849259 observations with 2113 users and 10084 items.
PROGRESS:     Data prepared in: 0.796311s
PROGRESS: Training ranking_factorization_recommender for recommendations.
PROGRESS: +------------------------------+--------------------------------------------------+----------+
PROGRESS: | Parameter                    | Description                                      | Value    |
PROGRESS: +------------------------------+--------------------------------------------------+----------+
PROGRESS: | num_factors                  | Factor Dimension                                 | 15       |
PROGRESS: | regularization               | L2 Regularization on Factors                     | 1e-009   |
PROGRESS: | max_iterations               | Maximum Number of Iterations                     | 100      |
PROGRESS: | solver                       | Solver used for training            

In [308]:
print 'RMSE:', model.evaluate_rmse(test_data, target='rating')['rmse_overall']

RMSE: 0.775337538264


Решателю ials снова не хватает итераций. Но качество уже значительно лучше, чем в случае квадратичных потерь.

А как влияет регуляризация?

In [309]:
train_data = prepare_data(train_df, binary=True) 
model = graphlab.ranking_factorization_recommender.create(train_data, target='rating',solver='adagrad',
                                                           num_factors=15, max_iterations=100, 
                                                           verbose=True, binary_target=True,
                                                           regularization = 1e-3)


PROGRESS: Recsys training: model = ranking_factorization_recommender
PROGRESS: Preparing data set.
PROGRESS:     Data has 849259 observations with 2113 users and 10084 items.
PROGRESS:     Data prepared in: 0.736513s
PROGRESS: Training ranking_factorization_recommender for recommendations.
PROGRESS: +--------------------------------+--------------------------------------------------+----------+
PROGRESS: | Parameter                      | Description                                      | Value    |
PROGRESS: +--------------------------------+--------------------------------------------------+----------+
PROGRESS: | num_factors                    | Factor Dimension                                 | 15       |
PROGRESS: | regularization                 | L2 Regularization on Factors                     | 0.001    |
PROGRESS: | solver                         | Solver used for training                         | adagrad  |
PROGRESS: | linear_regularization          | L2 Regularization on L

In [310]:
print 'RMSE:', model.evaluate_rmse(test_data, target='rating')['rmse_overall']

RMSE: 0.531227890484


Regularisation=1e-3 позволила снизить ошибку модели.

И как завершающий эксперимент с graphlab create, посмотрим как влияют на качество дополнительные признаки фильмов. Для этого нужно подготовить новую выборку.

In [316]:
movies_df.head()

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtAllCriticsRating,rtAllCriticsNumReviews,rtAllCriticsNumFresh,...,rtAllCriticsScore,rtTopCriticsRating,rtTopCriticsNumReviews,rtTopCriticsNumFresh,rtTopCriticsNumRotten,rtTopCriticsScore,rtAudienceRating,rtAudienceNumRatings,rtAudienceScore,rtPictureURL
0,1,Toy story,114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995,toy_story,9,73,73,...,100,8.5,17,17,0,100,3.7,102338,81,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995,1068044-jumanji,5.6,28,13,...,46,5.8,5,2,3,40,3.2,44587,61,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,107050,Dos viejos gru�ones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993,grumpy_old_men,5.9,36,24,...,66,7,6,5,1,83,3.2,10489,66,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995,waiting_to_exhale,5.6,25,14,...,56,5.5,11,5,6,45,3.3,5666,79,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,113041,Vuelve el padre de la novia (Ahora tambi�n abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995,father_of_the_bride_part_ii,5.3,19,9,...,47,5.4,5,1,4,20,3,13761,64,http://content8.flixster.com/movie/25/54/25542...


In [ ]:
frame_data = defaultdict(list)
movie_data = defaultdict(list)
for user, data in train_df.items():
    frame_data['item_id'] += list(data['movieID'])
    frame_data['rating'] += list(data['rating'])
    frame_data['user_id'] += ([user] * len(data['movieID']))
    
genres_df = pd.read_csv('data/movie_genres.dat', sep='\t')
genres_df.head()

for movie in frame_data['item_id']:
    data = movies_df[movies_df['id']==movie]
    movie_data['item_id'].append(movie)
    movie_data['title'].append(data['title'])
    movie_data['year'].append(data['year'])
    for genre in genres_df[genres_df['movieID']==movie]['genre']:
        movie_data['ganres'].append({movie:genre})




genres_df.head()

## Категориальные и разреженные признаки


Для этой части задания вам необходимо будет создать несколько выборок с разным набором признаков:
 - id-пользователя + id-фильма
 - id-пользователя + id-фильма + оценки пользователя за каждый просмотренный фильм
 - id-пользователя + id-фильма + жанры фильма
 - id-пользователя + id-фильма + жанры фильма + киноперсоны
 - набор признаков по вашему усмотрению (отличный от вышеперечисленных)
 
О том, как задавать выборки для конкретной библиотеки можно почитать в разделе "**В помощь**".

Со всеми полученными выборками проделайте следующие эксперименты:
 - Обучите **квадратичную модель** c помощью [Vopwal Wabbit](https://github.com/JohnLangford/vowpal_wabbit/wiki).
 - Постройте предсказания оценок при помощи факторизационных машин, используя библиотеку [LibFM](http://libfm.org). Попробуйте различные режимы работы: ALS, MCMC.
 
После этого:
 - Приведите сравнение качества всех моделей, используя две описанные метрики.
 - Какой набор признаков оказался более удачным? Как можно это обосновать?

## Vopwal Wabbit

### id-пользователя + id-фильма

In [8]:
def get_first_sample(df, file_name):
    results = []
    with open(file_name, 'w') as fsampl:
        for user, data in df.items():
            for row in data.iterrows():
                example = '{0} |f user:{1} movie:{2}\n'.format(row[1]['rating'], user, row[1]['movieID'])
                fsampl.write(example)
                results.append(row[1]['rating'])
        return results
    
get_first_sample(train_df, 'first_train.txt')
test_results = get_first_sample(test_df, 'first_test.txt')

In [9]:
!vw first_train.txt -q :: -f vw.model

creating quadratic features for pairs: :: 
You can use --leave_duplicate_interactions to disable this behaviour.
final_regressor = vw.model
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = first_train.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
16.000000 16.000000            1            1.0   4.0000   0.0000        6
8.000000 0.000000            2            2.0   4.0000   4.0000        6
6.798813 5.597626            4            4.0   4.0000   0.6916        6
4.705214 2.611616            8            8.0   3.5000   1.3001        6
3.395658 2.086102           16           16.0   2.0000   2.0602        6
2.325738 1.255818           32           32.0   3.0000   3.5978        6
1.465429 0.605121           64           64.0   4.0000   3.3284        6
1.076400 0.687371          128          128.0   4.0000   3.4549   

In [10]:
! vw  first_test.txt -t -q :: -p first_test_result.txt -i vw.model

creating quadratic features for pairs: :: 
You can use --leave_duplicate_interactions to disable this behaviour.
only testing
predictions = first_test_result.txt
Num weight bits = 18
learning rate = 10
initial_t = 1
power_t = 0.5
using no cache
Reading datafile = first_test.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.096111 0.096111            1            1.0   3.0000   3.3100        6
0.281105 0.466098            2            2.0   4.0000   3.3173        6
0.153014 0.024924            4            4.0   3.5000   3.3705        6
0.229506 0.305998            8            8.0   3.0000   3.4183        6
0.456294 0.683081           16           16.0   3.5000   3.3188        6
0.680011 0.903728           32           32.0   5.0000   3.6031        6
0.780809 0.881607           64           64.0   5.0000   3.3124        6
0.908739 1.036668          128          128.0   

Итак, теперь можно и измерить качество полученной модели:

In [11]:
def read_results(result_file):
    with open(result_file, 'r') as fin:
        results = fin.read().split()
    return map(float, results)
        
def calculate_RMSE(real_results, test_results):
    rmse = 0.0
    for real, test in zip(real_results, test_results):
        rmse += (real-test)**2
    return (rmse / len(real_results))**0.5

print 'RMSE: {:.3f}'.format(calculate_RMSE(test_results, read_results('first_test_result.txt')))

RMSE: 1.015


### id-пользователя + id-фильма + оценки пользователя за каждый просмотренный фильм

Если брать все оценки все пользователей, тренировочный файл получается огромный (до нескольких Gb). Так и не дождалась, пока модель на нем обучтся:( Возьмем только первые 10 рейтингов по величине, то есть будем учитывать только самые любимые фильмы каждого пользователя.

In [93]:
def get_first_sample(df, file_name):
    results = []
    with open(file_name, 'w') as fsampl:
        for user, data in df.items():
            ratings = ' '.join(['ranked-{0}:{1}'.format(movie,rating) for movie, rating in 
                                sorted(zip(list(data['movieID']), list(data['rating'])), 
                                       key=lambda x: x[1], reverse=True)[:10]])
            for row in data.iterrows():
                example = '{0} |f user:{1} movie:{2} {3}\n'.format(row[1]['rating'], user, int(row[1]['movieID']), ratings)
                fsampl.write(example)
                results.append(row[1]['rating'])
        return results
    
get_first_sample(train_df, 'second_train.txt')
test_results = get_first_sample(test_df, 'second_test.txt')

In [94]:
!vw second_train.txt -q :: -f vw.model

creating quadratic features for pairs: :: 
You can use --leave_duplicate_interactions to disable this behaviour.
final_regressor = vw.model
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = second_train.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
16.000000 16.000000            1            1.0   4.0000   0.0000       91
8.000000 0.000000            2            2.0   4.0000   4.0000       91
4.752882 1.505764            4            4.0   4.0000   2.3382       91
2.668049 0.583216            8            8.0   3.5000   2.2287       91
2.371038 2.074027           16           16.0   2.0000   2.2080       91
2.108972 1.846907           32           32.0   3.0000   3.7544       91
1.302356 0.495741           64           64.0   4.0000   2.9349       91
1.043651 0.784945          128          128.0   4.0000   3.7971  

In [95]:
! vw  second_test.txt -t -q :: -p second_test_result.txt -i vw.model

creating quadratic features for pairs: :: 
You can use --leave_duplicate_interactions to disable this behaviour.
only testing
predictions = second_test_result.txt
Num weight bits = 18
learning rate = 10
initial_t = 1
power_t = 0.5
using no cache
Reading datafile = second_test.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
4.646172 4.646172            1            1.0   3.0000   0.8445       21
7.291660 9.937148            2            2.0   4.0000   0.8477       21
5.968593 4.645526            4            4.0   3.5000   5.0000       21
5.265547 4.562500            8            8.0   3.0000   5.0000       21
4.759983 4.254420           16           16.0   3.5000   0.8860       21
7.036342 9.312700           32           32.0   5.0000   1.5436       21
6.754849 6.473357           64           64.0   5.0000   0.8931       21
6.745057 6.735265          128          128.0 

In [96]:
print 'RMSE: {:.3f}'.format(calculate_RMSE(test_results, read_results('second_test_result.txt')))

RMSE: 2.554


Судя по данной метрики, такая выборка не подходит для данной задачи.

### id-пользователя + id-фильма + жанры фильма

In [22]:
genres_df = pd.read_csv('data/movie_genres.dat', sep='\t')

In [107]:
def get_third_sample(df, file_name):
    results = []
    with open(file_name, 'w') as fsampl:
        for user, data in df.items():
            for row in data.iterrows():
                movie = int(row[1]['movieID'])
                genres = ' '.join([genre+':1.0' for genre in genres_df[genres_df['movieID']==movie]['genre']])
                example = '{0} |f user:{1} movie:{2} {3}\n'.format(row[1]['rating'], user, movie, genres)
                fsampl.write(example)
                results.append(row[1]['rating'])
        return results
    
get_third_sample(train_df, 'third_train.txt')
test_results = get_third_sample(test_df, 'third_test.txt')

In [108]:
!vw third_train.txt -q :: -f vw.model

creating quadratic features for pairs: :: 
You can use --leave_duplicate_interactions to disable this behaviour.
final_regressor = vw.model
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = third_train.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
16.000000 16.000000            1            1.0   4.0000   0.0000       21
8.000000 0.000000            2            2.0   4.0000   4.0000       28
4.062500 0.125000            4            4.0   4.0000   4.0000       36
3.445129 2.827758            8            8.0   3.5000   3.1284       45
3.320817 3.196505           16           16.0   2.0000   2.6723       28
2.550123 1.779429           32           32.0   3.0000   3.2398       28
2.027723 1.505324           64           64.0   4.0000   3.4296       28
1.467080 0.906437          128          128.0   4.0000   4.4414   

In [112]:
! vw  third_test.txt -t -q :: -p third_test_result.txt -i vw.model

creating quadratic features for pairs: :: 
You can use --leave_duplicate_interactions to disable this behaviour.
only testing
predictions = third_test_result.txt
Num weight bits = 18
learning rate = 10
initial_t = 1
power_t = 0.5
using no cache
Reading datafile = third_test.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.026550 0.026550            1            1.0   3.0000   3.1629       10
0.388303 0.750057            2            2.0   4.0000   3.1339       28
0.243537 0.098771            4            4.0   3.5000   3.3987       36
0.193888 0.144239            8            8.0   3.0000   3.0385       45
0.469969 0.746049           16           16.0   3.5000   3.8798       21
0.535967 0.601966           32           32.0   5.0000   3.8420       28
0.744027 0.952087           64           64.0   5.0000   3.7850       10
0.881777 1.019528          128          128.0   

In [113]:
print 'RMSE: {:.3f}'.format(calculate_RMSE(test_results, read_results('third_test_result.txt')))

RMSE: 0.998


Жанры повышают качество модели.

### id-пользователя + id-фильма + жанры фильма + киноперсоны

In [23]:
actors_df = pd.read_csv('data/movie_actors.dat', sep='\t') 

In [116]:
def get_fourth_sample(df, file_name):
    results = []
    with open(file_name, 'w') as fsampl:
        for user, data in df.items():
            for row in data.iterrows():
                movie = int(row[1]['movieID'])
                genres = ' '.join([genre+':1.0' for genre in genres_df[genres_df['movieID']==movie]['genre']])
                actors = ' '.join(['{}:1.0'.format(actor) for actor in actors_df[actors_df['movieID']==movie]['actorID']])
                example = '{0} |f user:{1} movie:{2} {3} {4}\n'.format(row[1]['rating'], user, movie, actors, genres)
                fsampl.write(example)
                results.append(row[1]['rating'])
        return results
    
get_fourth_sample(train_df, 'fourth_train.txt')
test_results = get_fourth_sample(test_df, 'fourth_test.txt')

In [117]:
!vw fourth_train.txt -q :: -f vw.model

creating quadratic features for pairs: :: 
You can use --leave_duplicate_interactions to disable this behaviour.
final_regressor = vw.model
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = fourth_train.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
16.000000 16.000000            1            1.0   4.0000   0.0000      231
8.000000 0.000000            2            2.0   4.0000   4.0000     1128
6.542069 5.084138            4            4.0   4.0000   0.8486      231
4.708262 2.874456            8            8.0   3.5000   4.0000      231
3.873983 3.039704           16           16.0   2.0000   0.1958      120
4.559501 5.245018           32           32.0   3.0000   5.0000     4851
5.204352 5.849202           64           64.0   4.0000   0.0000       91
6.337539 7.470726          128          128.0   4.0000   0.0000  

In [118]:
! vw  fourth_test.txt -t -q :: -p fourth_test_result.txt -i vw.model

creating quadratic features for pairs: :: 
You can use --leave_duplicate_interactions to disable this behaviour.
only testing
predictions = fourth_test_result.txt
Num weight bits = 18
learning rate = 10
initial_t = 1
power_t = 0.5
using no cache
Reading datafile = fourth_test.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.788322 0.788322            1            1.0   3.0000   3.8879     3160
0.589952 0.391583            2            2.0   4.0000   3.3742       45
0.718040 0.846128            4            4.0   3.5000   2.2037      171
0.615429 0.512818            8            8.0   3.0000   2.8281      300
0.706269 0.797109           16           16.0   3.5000   4.0778      465
0.717080 0.727892           32           32.0   5.0000   3.3696     2211
0.873704 1.030328           64           64.0   5.0000   3.3881       91
0.972330 1.070955          128          128.0 

In [119]:
print 'RMSE: {:.3f}'.format(calculate_RMSE(test_results, read_results('fourth_test_result.txt')))

RMSE: 1.016


Актеры никак не повлияли на качество модели. Можно обойтись и без них.

## id-пользователя + id-фильма + год + страна

In [13]:
country_df = pd.DataFrame.from_csv('data/movie_countries.dat', sep='\t')

In [18]:
def get_my_sample(df, file_name):
    results = []
    lines = []
    with open(file_name, 'w') as fsampl:
        for user, data in df.items():
            for row in data.iterrows():
                movie = int(row[1]['movieID'])
                country = country_df.loc[movie]['country']
                year = movies_df.iloc[movies[movie]]['year']
                example = '{0} |f user:{1} movie:{2} year:{3} {4}'.format(row[1]['rating'], user, 
                                                                   movie, year, country)
                lines.append(example)
                results.append(row[1]['rating'])
        fsampl.write('\n'.join(lines))
        return results
    
get_my_sample(train_df, 'my_train.txt')
test_results = get_my_sample(test_df, 'my_test.txt')

In [19]:
!vw my_train.txt -q :: -f vw.model

creating quadratic features for pairs: :: 
You can use --leave_duplicate_interactions to disable this behaviour.
final_regressor = vw.model
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = my_train.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
16.000000 16.000000            1            1.0   4.0000   0.0000       15
8.000000 0.000000            2            2.0   4.0000   4.0000       15
6.098874 4.197749            4            4.0   4.0000   1.1460       15
3.583344 1.067814            8            8.0   3.5000   2.0511       15
2.507625 1.431906           16           16.0   2.0000   2.5980       15
1.808298 1.108972           32           32.0   3.0000   4.0162       15
1.299444 0.790589           64           64.0   4.0000   3.2998       15
1.046178 0.792912          128          128.0   4.0000   3.5218      

In [20]:
! vw  my_test.txt -t -q :: -p my_test_result.txt -i vw.model

creating quadratic features for pairs: :: 
You can use --leave_duplicate_interactions to disable this behaviour.
only testing
predictions = my_test_result.txt
Num weight bits = 18
learning rate = 10
initial_t = 1
power_t = 0.5
using no cache
Reading datafile = my_test.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.103268 0.103268            1            1.0   3.0000   3.3214       15
0.268555 0.433841            2            2.0   4.0000   3.3413       15
0.149584 0.030614            4            4.0   3.5000   3.7473       21
0.238285 0.326985            8            8.0   3.0000   3.4107       15
0.456621 0.674957           16           16.0   3.5000   3.3414       15
0.651561 0.846501           32           32.0   5.0000   3.5660       15
0.739690 0.827819           64           64.0   5.0000   3.3145       15
0.872303 1.004917          128          128.0   4.5000

In [21]:
print 'RMSE: {:.3f}'.format(calculate_RMSE(test_results, read_results('my_test_result.txt')))

RMSE: 1.002


Страны и года немного повысили качество модели, но жанры все равно работают лучше.

## LIbFM

Теперь пришла очередь библиотеки LIbFM. Действовать будем аналогично. Только закодируем данные в соответствующем формате.

In [38]:
start_index = 0
movies_id = {}
for movie in movies_df['id'].unique():
    movies_id[movie] = len(movies_id) + start_index
start_index += len(movies_id)
users_id = {}
for user in rated_movies['userID'].unique():
    users_id[user] = len(users_id) + start_index
start_index +=len(users_id)

### id-пользователя + id-фильма

In [31]:
def get_first_sample(df, file_name):
    results = []
    with open(file_name, 'w') as fsampl:
        for user, data in df.items():
            for row in data.iterrows():
                example = '{0} {1}:1.0 {2}:1.0\n'.format(row[1]['rating'], users_id[user], movies_id[row[1]['movieID']])
                fsampl.write(example)
                results.append(row[1]['rating'])
        return results
    
get_first_sample(train_df, 'first_train.txt')
test_results = get_first_sample(test_df, 'first_test.txt')

In [80]:
!libFM.exe -task r -train first_train.txt -test first_test.txt -out first_result.txt -method mcmc

----------------------------------------------------------------------------
libFM
  Version: 1.40
  Author:  Steffen Rendle, steffen.rendle@uni-konstanz.de
  WWW:     http://www.libfm.org/
  License: Free for academic use. See license.txt.
----------------------------------------------------------------------------
Loading train...	
has x = 0
has xt = 1
num_rows=849259	num_values=1698518	num_features=12310	min_target=0.5	max_target=5
Loading test... 	
has x = 0
has xt = 1
num_rows=6339	num_values=12678	num_features=12310	min_target=0.5	max_target=5
#relations: 0
Loading meta data...	
#Iter=  0	Train=0.927721	Test=0.966066
#Iter=  1	Train=0.825262	Test=0.909677
#Iter=  2	Train=0.803978	Test=0.88292
#Iter=  3	Train=0.797963	Test=0.871065
#Iter=  4	Train=0.790678	Test=0.864292
#Iter=  5	Train=0.781681	Test=0.859693
#Iter=  6	Train=0.774546	Test=0.855662
#Iter=  7	Train=0.76886	Test=0.851957
#Iter=  8	Train=0.76242	Test=0.848541
#Iter=  9	Train=0.755235	Test=0.845441
#Iter= 10	Train=0.749

In [81]:
print 'RMSE: {:.3f}'.format(calculate_RMSE(test_results, read_results('first_result.txt')))

RMSE: 0.810


In [82]:
!libFM.exe -task r -train first_train.txt -test first_test.txt -out first_result.txt -method als

----------------------------------------------------------------------------
libFM
  Version: 1.40
  Author:  Steffen Rendle, steffen.rendle@uni-konstanz.de
  WWW:     http://www.libfm.org/
  License: Free for academic use. See license.txt.
----------------------------------------------------------------------------
Loading train...	
has x = 0
has xt = 1
num_rows=849259	num_values=1698518	num_features=12310	min_target=0.5	max_target=5
Loading test... 	
has x = 0
has xt = 1
num_rows=6339	num_values=12678	num_features=12310	min_target=0.5	max_target=5
#relations: 0
Loading meta data...	
#Iter=  0	Train=0.752214	Test=0.946868
#Iter=  1	Train=0.72344	Test=0.947484
#Iter=  2	Train=0.705624	Test=0.947459
#Iter=  3	Train=0.695947	Test=0.948411
#Iter=  4	Train=0.690154	Test=0.949674
#Iter=  5	Train=0.686248	Test=0.951213
#Iter=  6	Train=0.683417	Test=0.952617
#Iter=  7	Train=0.681268	Test=0.95368
#Iter=  8	Train=0.679577	Test=0.954526
#Iter=  9	Train=0.67821	Test=0.955211
#Iter= 10	Train=0.677

На базовой выборке имеем точность: <br>
<li>для метода MCMC RMSE = 0.81
<li>для метода ALS RMSE = 0.95

### id-пользователя + id-фильма + оценки пользователя за каждый просмотренный фильм

Снова будем рассматривать только самые высокие оценки пользователей.

In [39]:
movies_rank_id = {}
for movie in movies_df['id'].unique():
    movies_rank_id[movie] = len(movies_rank_id) + start_index
start_index += len(movies_rank_id)

In [16]:
def get_second_sample(df, file_name):
    results = []
    with open(file_name, 'w') as fsampl:
        for user, data in df.items():
            ratings = ' '.join(['{}:{}'.format(movies_rank_id[movie],rating) for movie, rating in 
                                       sorted(zip(list(data['movieID']), list(data['rating'])), 
                                       key=lambda x: x[1], reverse=True)[:10]])
            for row in data.iterrows():
                example = '{0} {1}:1.0 {2}:1.0 {3}\n'.format(row[1]['rating'], users_id[user], 
                                                                   movies_id[int(row[1]['movieID'])], ratings)
                results.append(row[1]['rating'])
        
                fsampl.write(example)
        return results
    
get_second_sample(train_df, 'second_train.txt')
test_results = get_second_sample(test_df, 'second_test.txt')

In [17]:
!libFM.exe -task r -train second_train.txt -test second_test.txt -out second_result.txt -method mcmc

----------------------------------------------------------------------------
libFM
  Version: 1.40
  Author:  Steffen Rendle, steffen.rendle@uni-konstanz.de
  WWW:     http://www.libfm.org/
  License: Free for academic use. See license.txt.
----------------------------------------------------------------------------
Loading train...	
has x = 0
has xt = 1
num_rows=849259	num_values=10191108	num_features=22481	min_target=0.5	max_target=5
Loading test... 	
has x = 0
has xt = 1
num_rows=6339	num_values=31695	num_features=22507	min_target=0.5	max_target=5
#relations: 0
Loading meta data...	
#Iter=  0	Train=1.05721	Test=1.67016
#Iter=  1	Train=0.818446	Test=1.61285
#Iter=  2	Train=0.79483	Test=1.59143
#Iter=  3	Train=0.7919	Test=1.58636
#Iter=  4	Train=0.790807	Test=1.57487
#Iter=  5	Train=0.790478	Test=1.5709
#Iter=  6	Train=0.789882	Test=1.56294
#Iter=  7	Train=0.789685	Test=1.56157
#Iter=  8	Train=0.789358	Test=1.55951
#Iter=  9	Train=0.789265	Test=1.55643
#Iter= 10	Train=0.789056	Test=1.

In [19]:
print 'RMSE: {:.3f}'.format(calculate_RMSE(test_results, read_results('second_result.txt')))

RMSE: 1.511
RMSE: 1.511


In [20]:
!libFM.exe -task r -train second_train.txt -test second_test.txt -out second_result.txt -method als

----------------------------------------------------------------------------
libFM
  Version: 1.40
  Author:  Steffen Rendle, steffen.rendle@uni-konstanz.de
  WWW:     http://www.libfm.org/
  License: Free for academic use. See license.txt.
----------------------------------------------------------------------------
Loading train...	
has x = 0
has xt = 1
num_rows=849259	num_values=10191108	num_features=22481	min_target=0.5	max_target=5
Loading test... 	
has x = 0
has xt = 1
num_rows=6339	num_values=31695	num_features=22507	min_target=0.5	max_target=5
#relations: 0
Loading meta data...	
#Iter=  0	Train=0.837151	Test=2.1793
#Iter=  1	Train=0.766909	Test=2.17627
#Iter=  2	Train=0.759375	Test=2.17686
#Iter=  3	Train=0.755865	Test=2.17804
#Iter=  4	Train=0.753175	Test=2.1799
#Iter=  5	Train=0.750819	Test=2.18183
#Iter=  6	Train=0.748715	Test=2.18335
#Iter=  7	Train=0.746846	Test=2.18497
#Iter=  8	Train=0.745171	Test=2.18642
#Iter=  9	Train=0.74364	Test=2.18769
#Iter= 10	Train=0.742227	Test=

Видим, что на данной выборке алгоритм работает хуже. Похоже, что он переобучается.: <br>
<li>для метода MCMC RMSE = 1.511
<li>для метода ALS RMSE = 2.20

### id-пользователя + id-фильма + жанры фильма

In [40]:
genres_id = {}
for genre in genres_df['genre'].unique():
    genres_id[genre] = len(genres_id) + start_index
start_index += len(genres_id)
print genres_id

{'Mystery': 22518, 'Drama': 22513, 'Western': 22525, 'Sci-Fi': 22519, 'Short': 22526, 'Horror': 22517, 'Film-Noir': 22524, 'Crime': 22515, 'Romance': 22512, 'Fantasy': 22511, 'Musical': 22523, 'Animation': 22508, 'War': 22522, 'Adventure': 22507, 'Action': 22514, 'Comedy': 22510, 'Documentary': 22521, 'Children': 22509, 'Thriller': 22516, 'IMAX': 22520}
{'Mystery': 22518, 'Drama': 22513, 'Western': 22525, 'Sci-Fi': 22519, 'Short': 22526, 'Horror': 22517, 'Film-Noir': 22524, 'Crime': 22515, 'Romance': 22512, 'Fantasy': 22511, 'Musical': 22523, 'Animation': 22508, 'War': 22522, 'Adventure': 22507, 'Action': 22514, 'Comedy': 22510, 'Documentary': 22521, 'Children': 22509, 'Thriller': 22516, 'IMAX': 22520}


In [31]:
grouped_df = genres_df.groupby('movieID')
def get_third_sample(df, file_name):
    results = []
    lines = []
    with open(file_name, 'w') as fsampl:
        for user, data in df.items():
            for row in data.iterrows():
                movie = int(row[1]['movieID'])
                genres = list(grouped_df.get_group(movie)['genre'])
                genres = ' '.join(['{}:1.0'.format(genres_id[genre]) for genre in genres])
                example = '{0} {1}:1.0 {2}:1.0 {3}'.format(row[1]['rating'], users_id[user], movies_id[movie], genres)
                #fsampl.write(example)
                lines.append(example)
                results.append(row[1]['rating'])
        fsampl.write('\n'.join(lines))
        return results
    
get_third_sample(train_df, 'third_train.txt')
test_results = get_third_sample(test_df, 'third_test.txt')

In [32]:
!libFM.exe -task r -train third_train.txt -test third_test.txt -out third_result.txt -method mcmc

----------------------------------------------------------------------------
libFM
  Version: 1.40
  Author:  Steffen Rendle, steffen.rendle@uni-konstanz.de
  WWW:     http://www.libfm.org/
  License: Free for academic use. See license.txt.
----------------------------------------------------------------------------
Loading train...	
has x = 0
has xt = 1
num_rows=849259	num_values=3921497	num_features=22527	min_target=0.5	max_target=5
Loading test... 	
has x = 0
has xt = 1
num_rows=6339	num_values=29914	num_features=22526	min_target=0.5	max_target=5
#relations: 0
Loading meta data...	
#Iter=  0	Train=0.914371	Test=0.946268
#Iter=  1	Train=0.809256	Test=0.894107
#Iter=  2	Train=0.789463	Test=0.873963
#Iter=  3	Train=0.778897	Test=0.862813
#Iter=  4	Train=0.771106	Test=0.855708
#Iter=  5	Train=0.764147	Test=0.850509
#Iter=  6	Train=0.758258	Test=0.846522
#Iter=  7	Train=0.7533	Test=0.843315
#Iter=  8	Train=0.748677	Test=0.840753
#Iter=  9	Train=0.745004	Test=0.838026
#Iter= 10	Train=0.74

In [33]:
print 'RMSE: {:.3f}'.format(calculate_RMSE(test_results, read_results('third_result.txt')))

RMSE: 0.808
RMSE: 0.808


In [35]:
!libFM.exe -task r -train third_train.txt -test third_test.txt -out third_result.txt -method als

----------------------------------------------------------------------------
libFM
  Version: 1.40
  Author:  Steffen Rendle, steffen.rendle@uni-konstanz.de
  WWW:     http://www.libfm.org/
  License: Free for academic use. See license.txt.
----------------------------------------------------------------------------
Loading train...	
has x = 0
has xt = 1
num_rows=849259	num_values=3921497	num_features=22527	min_target=0.5	max_target=5
Loading test... 	
has x = 0
has xt = 1
num_rows=6339	num_values=29914	num_features=22526	min_target=0.5	max_target=5
#relations: 0
Loading meta data...	
#Iter=  0	Train=0.761512	Test=0.917516
#Iter=  1	Train=0.730904	Test=0.91527
#Iter=  2	Train=0.71392	Test=0.915186
#Iter=  3	Train=0.704291	Test=0.917187
#Iter=  4	Train=0.69787	Test=0.919591
#Iter=  5	Train=0.693217	Test=0.922077
#Iter=  6	Train=0.689694	Test=0.924755
#Iter=  7	Train=0.686945	Test=0.927491
#Iter=  8	Train=0.68475	Test=0.929935
#Iter=  9	Train=0.682962	Test=0.93207
#Iter= 10	Train=0.68147

Жанры, не значительно, но все же повысили точность нашей модели:
<li>для метода MCMC RMSE = 0.808
<li>для метода ALS RMSE = 0.966

### id-пользователя + id-фильма + жанры фильма + киноперсоны

Будем рассматривать только актеров с самым высоким рейтингом.

In [41]:
actors_id = {}
for actor in actors_df['actorID'].unique():
    actors_id[actor] = len(actors_id) + start_index
start_index += len(actors_id)

In [53]:
actors_grouped_df = actors_df.groupby('movieID')
genres_grouped_df = genres_df.groupby('movieID')
def get_fourth_sample(df, file_name, prev_file_name):
    results = []
    lines = []
    f_prev = open(prev_file_name, 'r')
    with open(file_name, 'w') as fsampl:
        for user, data in df.items():
            for row in data.iterrows():
                prev = f_prev.readline().strip()
                movie = int(row[1]['movieID'])
                actors = list(actors_grouped_df.get_group(movie).sort('ranking', ascending=False)['actorID'])[:5] if movie in actors_grouped_df.groups.keys() else []
                actors = ' '.join(['{}:1.0'.format(actors_id[actor]) for actor in actors])
                lines.append(prev + ' ' + actors)
                results.append(row[1]['rating'])
        fsampl.write('\n'.join(lines))
        return results
    
get_fourth_sample(train_df, 'fourth_train.txt', 'third_train.txt')
test_results = get_fourth_sample(test_df, 'fourth_test.txt', 'third_test.txt')

In [54]:
!libFM.exe -task r -train fourth_train.txt -test fourth_test.txt -out fourth_result.txt -method mcmc

----------------------------------------------------------------------------
libFM
  Version: 1.40
  Author:  Steffen Rendle, steffen.rendle@uni-konstanz.de
  WWW:     http://www.libfm.org/
  License: Free for academic use. See license.txt.
----------------------------------------------------------------------------
Loading train...	
has x = 0
has xt = 1
num_rows=849259	num_values=8086490	num_features=117843	min_target=0.5	max_target=5
Loading test... 	
has x = 0
has xt = 1
num_rows=6339	num_values=60952	num_features=117848	min_target=0.5	max_target=5
#relations: 0
Loading meta data...	
#Iter=  0	Train=0.890209	Test=0.940033
#Iter=  1	Train=0.800588	Test=0.889331
#Iter=  2	Train=0.783927	Test=0.873686
#Iter=  3	Train=0.769235	Test=0.865875
#Iter=  4	Train=0.759988	Test=0.859156
#Iter=  5	Train=0.754087	Test=0.854297
#Iter=  6	Train=0.748968	Test=0.850454
#Iter=  7	Train=0.745354	Test=0.84727
#Iter=  8	Train=0.742148	Test=0.844761
#Iter=  9	Train=0.739311	Test=0.842449
#Iter= 10	Train=0

In [55]:
print 'RMSE: {:.3f}'.format(calculate_RMSE(test_results, read_results('fourth_result.txt')))

RMSE: 0.811
RMSE: 0.811


In [56]:
!libFM.exe -task r -train fourth_train.txt -test fourth_test.txt -out fourth_result.txt -method als

----------------------------------------------------------------------------
libFM
  Version: 1.40
  Author:  Steffen Rendle, steffen.rendle@uni-konstanz.de
  WWW:     http://www.libfm.org/
  License: Free for academic use. See license.txt.
----------------------------------------------------------------------------
Loading train...	
has x = 0
has xt = 1
num_rows=849259	num_values=8086490	num_features=117843	min_target=0.5	max_target=5
Loading test... 	
has x = 0
has xt = 1
num_rows=6339	num_values=60952	num_features=117848	min_target=0.5	max_target=5
#relations: 0
Loading meta data...	
#Iter=  0	Train=0.754642	Test=0.913609
#Iter=  1	Train=0.724434	Test=0.913127
#Iter=  2	Train=0.707572	Test=0.913302
#Iter=  3	Train=0.697956	Test=0.913994
#Iter=  4	Train=0.691714	Test=0.915314
#Iter=  5	Train=0.687271	Test=0.916584
#Iter=  6	Train=0.683934	Test=0.91795
#Iter=  7	Train=0.68135	Test=0.919196
#Iter=  8	Train=0.679281	Test=0.920453
#Iter=  9	Train=0.677589	Test=0.921552
#Iter= 10	Train=0.

Актеры не оказали особого влияния на качество работы алгоритма. Может, мы взяли мало топовых актеров. Но формирование выборки и тренировка и занимают много времени. Так что остановимся на этом:
<li>для метода MCMC RMSE = 0.811
<li>для метода ALS RMSE = 0.958

### id-пользователя + id-фильма + год + страна

In [ ]:
country_df = pd.DataFrame.from_csv('data/movie_countries.dat', sep='\t')

In [75]:
country_id = {}
for country in country_df['country'].unique():
    country_id[country] = len(country_id) + start_index
start_index += len(country_id)

year_id = start_index
start_index += 1

In [76]:
def get_my_sample(df, file_name):
    results = []
    lines = []
    with open(file_name, 'w') as fsampl:
        for user, data in df.items():
            for row in data.iterrows():
                movie = int(row[1]['movieID'])
                country = country_id[country_df.loc[movie]['country']]
                year = movies_df.iloc[movies_id[movie]]['year']
                example = '{0} {1}:1.0 {2}:1.0 {3}:{4} {5}:1.0'.format(row[1]['rating'], users_id[user], 
                                                                   movies_id[movie], year_id, year, country)
                lines.append(example)
                results.append(row[1]['rating'])
        fsampl.write('\n'.join(lines))
        return results
    
get_my_sample(train_df, 'my_train.txt')
test_results = get_my_sample(test_df, 'my_test.txt')

In [77]:
!libFM.exe -task r -train my_train.txt -test my_test.txt -out my_result.txt -method mcmc

----------------------------------------------------------------------------
libFM
  Version: 1.40
  Author:  Steffen Rendle, steffen.rendle@uni-konstanz.de
  WWW:     http://www.libfm.org/
  License: Free for academic use. See license.txt.
----------------------------------------------------------------------------
Loading train...	
has x = 0
has xt = 1
num_rows=849259	num_values=3397036	num_features=117921	min_target=0.5	max_target=5
Loading test... 	
has x = 0
has xt = 1
num_rows=6339	num_values=25356	num_features=117921	min_target=0.5	max_target=5
#relations: 0
Loading meta data...	
#Iter=  0	Train=2.30111	Test=2.38927
#Iter=  1	Train=1.22182	Test=1.54964
#Iter=  2	Train=0.813961	Test=1.2177
#Iter=  3	Train=0.800445	Test=1.07492
#Iter=  4	Train=0.800264	Test=1.00136
#Iter=  5	Train=0.800202	Test=0.960803
#Iter=  6	Train=0.800341	Test=0.93628
#Iter=  7	Train=0.800257	Test=0.9196
#Iter=  8	Train=0.800242	Test=0.9085
#Iter=  9	Train=0.800234	Test=0.900307
#Iter= 10	Train=0.800372	Test

In [78]:
print 'RMSE: {:.3f}'.format(calculate_RMSE(test_results, read_results('my_result.txt')))

RMSE: 0.867


In [79]:
!libFM.exe -task r -train my_train.txt -test my_test.txt -out my_result.txt -method als

----------------------------------------------------------------------------
libFM
  Version: 1.40
  Author:  Steffen Rendle, steffen.rendle@uni-konstanz.de
  WWW:     http://www.libfm.org/
  License: Free for academic use. See license.txt.
----------------------------------------------------------------------------
Loading train...	
has x = 0
has xt = 1
num_rows=849259	num_values=3397036	num_features=117921	min_target=0.5	max_target=5
Loading test... 	
has x = 0
has xt = 1
num_rows=6339	num_values=25356	num_features=117921	min_target=0.5	max_target=5
#relations: 0
Loading meta data...	
#Iter=  0	Train=0.974735	Test=1.07504
#Iter=  1	Train=0.97472	Test=1.07503
#Iter=  2	Train=0.974704	Test=1.07502
#Iter=  3	Train=0.974689	Test=1.07501
#Iter=  4	Train=0.974674	Test=1.07501
#Iter=  5	Train=0.974659	Test=1.075
#Iter=  6	Train=0.974643	Test=1.07499
#Iter=  7	Train=0.974628	Test=1.07498
#Iter=  8	Train=0.974613	Test=1.07498
#Iter=  9	Train=0.974598	Test=1.07497
#Iter= 10	Train=0.974583	Test

Вывод: год и страна фильма не помогли повысить точность.
<li>для метода MCMC RMSE = 0.867
<li>для метода ALS RMSE = 1.0743

Вывод: LIbFM работает быстрее, чем Vowal Wabbit, и позволяет добиться лучшего качества. Наиболее эффективная выборка состоит из id пользователей и фильмов и жанров. 